This notebook demonstrates how to concatenate two CSV files, standardize column names, and ingest the resulting data into a SQL Server database running on Docker. We will be using Pandas for data manipulation and pymssql for interacting with the SQL Server.

Prerequistes
Befor running this notebook, ensure that you have the following installed:
- Python
- Jupyter Notebook
- Docker
- SQL Server Docker container running with ports exposed : https://hub.docker.com/r/microsoft/mssql-server
- Necessary Python libraries:
    - pandas
    - pymssql 

In [1]:
import pymssql
import pandas as pd
import numpy as np

In [2]:
server = 'localhost'
database = 'master'
username = 'sa' 
password = 'yourStrong(!)Passowrd'

In [17]:
try:
    conn = pymssql.connect(server=server,  user=username, password=password, database=database,autocommit=True)

    print("connection successful")
except Exception as e:
    print(f"Error : {e}")


connection successful


In [18]:
cursor = conn.cursor()

In [19]:
cursor.execute("""
               IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'crimedatabase')
               BEGIN
                    CREATE DATABASE crimedatabase;
               END""")

In [23]:
data_10_19 = pd.read_csv('C:/Users/Varun/Downloads/Crime_Data_from_2010_to_2019.csv')

In [24]:
data_10_19.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [29]:
data_20_24 = pd.read_csv('C:/Users/Varun/Downloads/Crime_Data_from_2020_to_present.csv')

In [31]:
data_20_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123044 entries, 0 to 2123043
Data columns (total 28 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DR_NO           int64  
 1   Date Rptd       object 
 2   DATE OCC        object 
 3   TIME OCC        int64  
 4   AREA            int64  
 5   AREA NAME       object 
 6   Rpt Dist No     int64  
 7   Part 1-2        int64  
 8   Crm Cd          int64  
 9   Crm Cd Desc     object 
 10  Mocodes         object 
 11  Vict Age        int64  
 12  Vict Sex        object 
 13  Vict Descent    object 
 14  Premis Cd       float64
 15  Premis Desc     object 
 16  Weapon Used Cd  float64
 17  Weapon Desc     object 
 18  Status          object 
 19  Status Desc     object 
 20  Crm Cd 1        float64
 21  Crm Cd 2        float64
 22  Crm Cd 3        float64
 23  Crm Cd 4        float64
 24  LOCATION        object 
 25  Cross Street    object 
 26  LAT             float64
 27  LON             float64
dtypes: float64(8

In [32]:
data_20_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982638 entries, 0 to 982637
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           982638 non-null  int64  
 1   Date Rptd       982638 non-null  object 
 2   DATE OCC        982638 non-null  object 
 3   TIME OCC        982638 non-null  int64  
 4   AREA            982638 non-null  int64  
 5   AREA NAME       982638 non-null  object 
 6   Rpt Dist No     982638 non-null  int64  
 7   Part 1-2        982638 non-null  int64  
 8   Crm Cd          982638 non-null  int64  
 9   Crm Cd Desc     982638 non-null  object 
 10  Mocodes         837376 non-null  object 
 11  Vict Age        982638 non-null  int64  
 12  Vict Sex        844193 non-null  object 
 13  Vict Descent    844182 non-null  object 
 14  Premis Cd       982624 non-null  float64
 15  Premis Desc     982053 non-null  object 
 16  Weapon Used Cd  326167 non-null  float64
 17  Weapon Des

In [46]:
complete_data = pd.concat([data_10_19,data_20_24],axis=0,ignore_index=True)

In [47]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 29 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DR_NO           int64  
 1   Date Rptd       object 
 2   DATE OCC        object 
 3   TIME OCC        int64  
 4   AREA            float64
 5   AREA NAME       object 
 6   Rpt Dist No     int64  
 7   Part 1-2        int64  
 8   Crm Cd          int64  
 9   Crm Cd Desc     object 
 10  Mocodes         object 
 11  Vict Age        int64  
 12  Vict Sex        object 
 13  Vict Descent    object 
 14  Premis Cd       float64
 15  Premis Desc     object 
 16  Weapon Used Cd  float64
 17  Weapon Desc     object 
 18  Status          object 
 19  Status Desc     object 
 20  Crm Cd 1        float64
 21  Crm Cd 2        float64
 22  Crm Cd 3        float64
 23  Crm Cd 4        float64
 24  LOCATION        object 
 25  Cross Street    object 
 26  LAT             float64
 27  LON             float64
 28  AREA        

In [53]:
complete_data.rename(columns={'DR_NO':'file_no',
                              'Date Rptd':'date_rptd',
                              'DATE OCC':'date_occ',
                              'TIME OCC':'time_occ',
                              'AREA ':'area_code',
                              'AREA NAME':'area_name',
                              'Rpt Dist No':'rpt_dist_no',
                              'Crm Cd':'crm_code',
                              'Crm Cd Desc':'crm_code_description',
                              'Mocodes':'modus_operandi',
                              'Vict Age':'victim_age',
                              'Vict Sex':'victim_sex',
                              'Vict Descent':'victim_descent',
                              'Premis Cd':'premise_code',
                              'Premis Desc':'premise_code_desc',
                              'Weapon Used Cd':'weapon_used_code',
                              'Weapon Desc':'weapon_desc',
                              'Status':'status_code',
                              'Status Desc':'status_desc',
                              'Crm Cd 1':'crm_code_1',
                              'Crm Cd 2':'crm_code_2',
                              'Crm Cd 3':'crm_code_3',
                              'Crm Cd 4':'crm_code_4',
                              'LOCATION':'crm_location',
                              'Cross Street':'cross_street',
                              'LAT':'latitude',
                              'LON':'longitude'},inplace=True)

In [54]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   file_no               int64  
 1   date_rptd             object 
 2   date_occ              object 
 3   time_occ              int64  
 4   area_code             float64
 5   area_name             object 
 6   rpt_dist_no           int64  
 7   crm_code              int64  
 8   crm_code_description  object 
 9   modus_operandi        object 
 10  victim_age            int64  
 11  victim_sex            object 
 12  victim_descent        object 
 13  premise_code          float64
 14  premise_code_desc     object 
 15  weapon_used_code      float64
 16  weapon_desc           object 
 17  status_code           object 
 18  status_desc           object 
 19  crm_code_1            float64
 20  crm_code_2            float64
 21  crm_code_3            float64
 22  crm_code_4            float64
 23  crm_loc

In [ ]:
complete_data.drop(columns={'Part 1-2','AREA'},inplace=True)
complete_data.replace({np.nan:None},inplace=True)

In [51]:
complete_data

,file_no,date_rptd,date_occ,time_occ,area_code,area_name,rpt_dist_no,crm_code,crm_code_description,modus_operandi,...,status_code,status_desc,crm_code_1,crm_code_2,crm_code_3,crm_code_4,crm_location,cross_street,latitude,longitude
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13.0,Newton,1385,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14.0,Pacific,1485,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13.0,Newton,1324,946,OTHER MISCELLANEOUS CRIME,0344,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6.0,Hollywood,646,900,VIOLATION OF COURT ORDER,1100 0400 1402,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1.0,Central,176,122,"RAPE, ATTEMPTED",0400,...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105677,242011172,08/20/2024 12:00:00 AM,08/17/2024 12:00:00 AM,2300,NaN,Olympic,2033,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0344 1822,...,IC,Invest Cont,341.0,NaN,NaN,NaN,3700 WILSHIRE BL,NaN,34.0617,-118.3066
3105678,240710284,07/24/2024 12:00:00 AM,07/23/2024 12:00:00 AM,1400,NaN,Wilshire,788,510,VEHICLE - STOLEN,NaN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,4000 W 23RD ST,NaN,34.0362,-118.3284
3105679,240104953,01/15/2024 12:00:00 AM,01/15/2024 12:00:00 AM,100,NaN,Central,101,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 0400 0416,...,IC,Invest Cont,745.0,NaN,NaN,NaN,1300 W SUNSET BL,NaN,34.0685,-118.2460
3105680,240309674,04/24/2024 12:00:00 AM,04/24/2024 12:00:00 AM,1500,NaN,Southwest,358,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",1822 0334 0416 0445 0449 1202,...,IC,Invest Cont,230.0,NaN,NaN,NaN,FLOWER ST,JEFFERSON BL,34.0215,-118.2868


In [56]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   file_no               int64  
 1   date_rptd             object 
 2   date_occ              object 
 3   time_occ              int64  
 4   area_code             float64
 5   area_name             object 
 6   rpt_dist_no           int64  
 7   crm_code              int64  
 8   crm_code_description  object 
 9   modus_operandi        object 
 10  victim_age            int64  
 11  victim_sex            object 
 12  victim_descent        object 
 13  premise_code          float64
 14  premise_code_desc     object 
 15  weapon_used_code      float64
 16  weapon_desc           object 
 17  status_code           object 
 18  status_desc           object 
 19  crm_code_1            float64
 20  crm_code_2            float64
 21  crm_code_3            float64
 22  crm_code_4            float64
 23  crm_loc

In [75]:
cursor.execute("""
               IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'crimedata' AND TABLE_SCHEMA='dbo')

                BEGIN
                      CREATE  TABLE crimedatabase.dbo.crimedata (
                                                                                  file_no int PRIMARY KEY,
                                                                                  date_rptd date,
                                                                                  date_occ date,
                                                                                  time_occ int,
                                                                                  area_code float,
                                                                                  area_name varchar(255),
                                                                                  rpt_dist_no int,
                                                                                  crm_code int,
                                                                                  crm_code_description varchar(255),
                                                                                  modus_operandi varchar(255),
                                                                                  victim_age int,
                                                                                  victim_sex varchar(10),
                                                                                  vitim_descent varchar(100),
                                                                                  premise_code float,
                                                                                  premise_code_desc varchar(255),
                                                                                  weapon_used_code float,
                                                                                  weapon_desc varchar(255),
                                                                                  status_code varchar(100),
                                                                                  status_desc varchar(255),
                                                                                  crm_code_1 float,
                                                                                  crm_code_2 float,
                                                                                  crm_code_3 float,
                                                                                  crm_code_4 float,
                                                                                  crm_location varchar(255),
                                                                                  cross_street varchar(255),
                                                                                  latitude float,
                                                                                  longitude float                    
                                                                                  )
                END
                """)

In [76]:
row_count = 0
for index, row in complete_data.iterrows():
    row_count +=1
    if row_count%100000 == 0:
        print('Row entered: ',row_count)
    cursor.execute("""
                        INSERT INTO crimedatabase.dbo.crimedata (
                                                                    file_no,
                                                                    date_rptd,
                                                                    date_occ,
                                                                    time_occ,
                                                                    area_code,
                                                                    area_name,
                                                                    rpt_dist_no,
                                                                    crm_code,
                                                                    crm_code_description,
                                                                    modus_operandi,
                                                                    victim_age,
                                                                    victim_sex,
                                                                    vitim_descent,
                                                                    premise_code,
                                                                    premise_code_desc,
                                                                    weapon_used_code,
                                                                    weapon_desc,
                                                                    status_code,
                                                                    status_desc,
                                                                    crm_code_1,
                                                                    crm_code_2,
                                                                    crm_code_3,
                                                                    crm_code_4,
                                                                    crm_location,
                                                                    cross_street,
                                                                    latitude,
                                                                    longitude                    
                                                                )
                        VALUES (    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s)
                            """,
                            (
                                row['file_no'],row['date_rptd'],row['date_occ'],row['time_occ'],row['area_code'],row['area_name'],row['rpt_dist_no'],
                                row['crm_code'],row['crm_code_description'],row['modus_operandi'],row['victim_age'],row['victim_sex'],row['victim_descent'],
                                row['premise_code'],row['premise_code_desc'],row['weapon_used_code'],row['weapon_desc'],row['status_code'],row['status_desc'],
                                row['crm_code_1'],row['crm_code_2'],row['crm_code_3'],row['crm_code_4'],row['crm_location'],row['cross_street'],row['latitude'],row['longitude']
                            )
                        )

Row entered:  100000
Row entered:  200000
Row entered:  300000
Row entered:  400000
Row entered:  500000
Row entered:  600000
Row entered:  700000
Row entered:  800000
Row entered:  900000
Row entered:  1000000
Row entered:  1100000
Row entered:  1200000
Row entered:  1300000
Row entered:  1400000
Row entered:  1500000
Row entered:  1600000
Row entered:  1700000
Row entered:  1800000
Row entered:  1900000
Row entered:  2000000
Row entered:  2100000
Row entered:  2200000
Row entered:  2300000
Row entered:  2400000
Row entered:  2500000
Row entered:  2600000
Row entered:  2700000
Row entered:  2800000
Row entered:  2900000
Row entered:  3000000
Row entered:  3100000


In [65]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   file_no               int64  
 1   date_rptd             object 
 2   date_occ              object 
 3   time_occ              int64  
 4   area_code             float64
 5   area_name             object 
 6   rpt_dist_no           int64  
 7   crm_code              int64  
 8   crm_code_description  object 
 9   modus_operandi        object 
 10  victim_age            int64  
 11  victim_sex            object 
 12  victim_descent        object 
 13  premise_code          float64
 14  premise_code_desc     object 
 15  weapon_used_code      float64
 16  weapon_desc           object 
 17  status_code           object 
 18  status_desc           object 
 19  crm_code_1            float64
 20  crm_code_2            float64
 21  crm_code_3            float64
 22  crm_code_4            float64
 23  crm_loc